In [2]:
# Download the dataset from Kaggle using the Kaggle API

import opendatasets as od

# Eric's Kaggle API key
# "username":"ericthedataguy",
# "key":"875c0de0faea2fefa081c78eb470a347"

url_cleaned = 'https://www.kaggle.com/datasets/ericthedataguy/cleaned-hotel-review-for-ba820'

od.download(url_cleaned)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading cleaned-hotel-review-for-ba820.zip to ./cleaned-hotel-review-for-ba820


100%|██████████| 45.1M/45.1M [00:01<00:00, 31.4MB/s]


In [4]:
# Read the dataset into a pandas dataframe

import pandas as pd

filepath = '/Users/hakukazuho/Luxurious-Hotel-Review-1/cleaned-hotel-review-for-ba820/hotel_reviews_cleaned.csv'
df = pd.read_csv(filepath)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   Positive_Review                             515738 non-null  object 
 

In [5]:
# Drop outliers in the 'Reviewer_Score' column

# Calculate the IQR
Q1 = df['Reviewer_Score'].quantile(0.25)
Q3 = df['Reviewer_Score'].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Drop the outliers and store the results in df_drop
df_drop = df[(df['Reviewer_Score'] >= lower_bound) & (df['Reviewer_Score'] <= upper_bound)]
df_drop.shape[0]


499831

In [10]:
df_drop['Positive_Review'].value_counts(ascending=True).head(10)

Positive_Review
No real complaints the hotel was great great location surroundings rooms amenities and service Two recommendations however firstly the staff upon check in are very confusing regarding deposit payments and the staff offer you upon checkout to refund your original payment and you can make a new one Bit confusing Secondly the on site restaurant is a bit lacking very well thought out and excellent quality food for anyone of a vegetarian or vegan background but even a wrap or toasted sandwich option would be great Aside from those minor minor things fantastic spot and will be back when i return to Amsterdam     1
The staff were nice and the room was nicely sized with reasonably priced minibar items                                                                                                                                                                                                                                                                                          

In [11]:
df_drop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499831 entries, 1 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               499831 non-null  object 
 1   Additional_Number_of_Scoring                499831 non-null  int64  
 2   Review_Date                                 499831 non-null  object 
 3   Average_Score                               499831 non-null  float64
 4   Hotel_Name                                  499831 non-null  object 
 5   Reviewer_Nationality                        499831 non-null  object 
 6   Negative_Review                             499831 non-null  object 
 7   Review_Total_Negative_Word_Counts           499831 non-null  int64  
 8   Total_Number_of_Reviews                     499831 non-null  int64  
 9   Positive_Review                             499831 non-null  object 
 10  R

In [17]:
# Create subset for reccomendation system 1.0

df2_col_list = ['Hotel_Name', 'Negative_Review', 'Positive_Review', 'Tags']
df2 = df_drop[df2_col_list].iloc[:1000]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 1 to 1034
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Hotel_Name       1000 non-null   object
 1   Negative_Review  1000 non-null   object
 2   Positive_Review  1000 non-null   object
 3   Tags             1000 non-null   object
dtypes: object(4)
memory usage: 39.1+ KB


In [18]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

# Tokenize the Negative_Review column into sentences
df2['Negative_Review'] = df2['Negative_Review'].apply(lambda x: sent_tokenize(x))

# Tokenize the Positive_Review column into sentences
df2['Positive_Review'] = df2['Positive_Review'].apply(lambda x: sent_tokenize(x))

# Tokenize the Negative_Review column into words
df2['Negative_Review'] = df2['Negative_Review'].apply(lambda x: [word_tokenize(sentence) for sentence in x])

# Tokenize the Positive_Review column into words
df2['Positive_Review'] = df2['Positive_Review'].apply(lambda x: [word_tokenize(sentence) for sentence in x])

df2.head()


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hakukazuho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Hotel_Name,Negative_Review,Positive_Review,Tags
1,Hotel Arena,"[[No, Negative]]","[[No, real, complaints, the, hotel, was, great...","[' Leisure trip ', ' Couple ', ' Duplex Double..."
2,Hotel Arena,"[[Rooms, are, nice, but, for, elderly, a, bit,...","[[Location, was, good, and, staff, were, ok, I...","[' Leisure trip ', ' Family with young childre..."
4,Hotel Arena,"[[You, When, I, booked, with, your, company, o...","[[Amazing, location, and, building, Romantic, ...","[' Leisure trip ', ' Couple ', ' Suite ', ' St..."
5,Hotel Arena,"[[Backyard, of, the, hotel, is, total, mess, s...","[[Good, restaurant, with, modern, design, grea...","[' Leisure trip ', ' Group ', ' Duplex Double ..."
6,Hotel Arena,"[[Cleaner, did, not, change, our, sheet, and, ...","[[The, room, is, spacious, and, bright, The, h...","[' Leisure trip ', ' Group ', ' Duplex Twin Ro..."


In [21]:
# Function to flatten a list of lists
def flatten_list_of_lists(list_of_lists):
    return [item for sublist in list_of_lists for item in sublist]

# Apply the function to each row in the DataFrame column
df2['Negative_Review'] = df2['Negative_Review'].apply(flatten_list_of_lists)
df2['Positive_Review'] = df2['Positive_Review'].apply(flatten_list_of_lists)
df2.head()

,Hotel_Name,Negative_Review,Positive_Review,Tags
1,Hotel Arena,"[No, Negative]","[No, real, complaints, the, hotel, was, great,...","[' Leisure trip ', ' Couple ', ' Duplex Double..."
2,Hotel Arena,"[Rooms, are, nice, but, for, elderly, a, bit, ...","[Location, was, good, and, staff, were, ok, It...","[' Leisure trip ', ' Family with young childre..."
4,Hotel Arena,"[You, When, I, booked, with, your, company, on...","[Amazing, location, and, building, Romantic, s...","[' Leisure trip ', ' Couple ', ' Suite ', ' St..."
5,Hotel Arena,"[Backyard, of, the, hotel, is, total, mess, sh...","[Good, restaurant, with, modern, design, great...","[' Leisure trip ', ' Group ', ' Duplex Double ..."
6,Hotel Arena,"[Cleaner, did, not, change, our, sheet, and, d...","[The, room, is, spacious, and, bright, The, ho...","[' Leisure trip ', ' Group ', ' Duplex Twin Ro..."


### What's Next?

1. Tags and Review have different format

2. Extract keywords from Reviews

3. Store keywords to negative/positive tag pools